In [5]:
n=3

In [6]:
G = SymmetricGroup(n)

In [7]:
SGA = G.algebra(QQ)

In [8]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [93]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [9]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick: 
#define a new inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis using this inner product

In [54]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,unitary=False):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    rho = specht_module.representation_matrix
    d_part = specht_module.dimension()
    if unitary:
        M = unitary_change_of_basis(partition)
        return sqrt(d_part/G.order())*sum(f(g)*M.inverse()*rho(g)*M for g in G)
    else:
        return sum(f(g)*rho(g) for g in G)

In [53]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [52]:
#for each basis element g \in G
#compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
#make the Fourier coefficients into a list via .list()
#flatten the list of lists to get a vector \hat{g}
#put the vectors into a list and take the transpose to get the DFT
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition,unitary=True).list() for partition in Partitions(n)]) for g in G]).transpose()

In [51]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(partition):
    specht_module = G.algebra(QQ).specht_module(partition)
    rho = specht_module.representation_matrix
    P = (1/G.order())*sum(rho(g)*rho(g).conjugate().transpose() for g in G)
    Q = P.principal_square_root()
    return Q

In [45]:
#ISSUE: the change-of-basis matrix is not making the representations unitary
#SOLUTION: use the simpler form of the change-of-basis 
#which just averages the repsentations "squared" over the group and takes a square root

In [55]:
Q = unitary_change_of_basis(partition); Q

[1/2*sqrt(2) + 1/2*sqrt(2/3) 1/2*sqrt(2) - 1/2*sqrt(2/3)]
[1/2*sqrt(2) - 1/2*sqrt(2/3) 1/2*sqrt(2) + 1/2*sqrt(2/3)]

In [49]:
U = (Q.inverse()*rho(G[5])*Q).simplify_full(); (U*U.transpose()).simplify_full()

[1 0]
[0 1]

In [56]:
partition = Partitions(n)[1]

In [57]:
specht_module = G.algebra(QQ).specht_module(partition)

In [58]:
rho = specht_module.representation_matrix

In [59]:
d_part = specht_module.dimension()

In [61]:
#check that the DFT is unitary
(unitary_dft(3)*unitary_dft(3).transpose()).simplify_full()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [58]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [92]:
eigs = A.eigenvalues(); eigs

[-1.927382440195288?, 2.121715010353017?, -1.174423554578382? - 1.623003052198150?*I, -1.174423554578382? + 1.623003052198150?*I, 1.452257269499517? - 1.086817272076326?*I, 1.452257269499517? + 1.086817272076326?*I]

In [93]:
[abs(eig) for eig in eigs]

[1.927382440195288?,
 2.121715010353017?,
 2.003349593304431?,
 2.003349593304431?,
 1.813897174510622?,
 1.813897174510622?]

In [121]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.44948974 1.73205081 1.5        2.         1.73205081 2.44948974]
